In [45]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [46]:
from util import initialize
AI_MODEL = initialize()

from pprint import pprint
from typing import List, Dict
from pprint import pprint, pformat

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['openai:gpt-4o',
 'openai:gpt-4o-mini',
 'gemini-1.5-pro',
 'gemini-2.0-flash-exp',
 'claude-3-5-haiku-latest',
 'claude-3-5-sonnet-latest']
Using AI model: openai:gpt-4o


In [47]:
class GeneratorResponse(BaseModel):
    thoughts: str = Field(..., description='Your understanding of the task and feedback and how you plan to improve')
    response: str = Field(..., description="The generated solution.")


async def generate(prompt: str, task: str, context: str = "") -> tuple[str, str]:
    """Generate and improve a solution based on feedback."""
    full_prompt = f"{prompt}\n{context}\nTask: {task}" if context else f"{prompt}\nTask: {task}"
    response = (await Agent(AI_MODEL, result_type=GeneratorResponse).run(full_prompt)).data
    thoughts = response.thoughts
    result = response.response
    
    print("\n=== GENERATION START ===")
    print(f"Thoughts:\n{thoughts}\n")
    print(f"Generated:\n{result}")
    print("=== GENERATION END ===\n")
    
    return thoughts, result


class EvaluatorResponse(BaseModel):
    evaluation: str = Field(..., description='PASS, NEEDS_IMPROVEMENT, or FAIL')
    feedback: str = Field(..., description='What needs improvement and why.')


async def evaluate(prompt: str, content: str, task: str) -> tuple[str, str]:
    """Evaluate if a solution meets requirements."""
    full_prompt = f"{prompt}\nOriginal task: {task}\nContent to evaluate: {content}"
    response = (await Agent(AI_MODEL, result_type=EvaluatorResponse).run(full_prompt)).data
    evaluation = response.evaluation
    feedback = response.feedback
    
    print("=== EVALUATION START ===")
    print(f"Status: {evaluation}")
    print(f"Feedback: {feedback}")
    print("=== EVALUATION END ===\n")
    
    return evaluation, feedback


async def loop(task: str, evaluator_prompt: str, generator_prompt: str) -> tuple[str, list[dict]]:
    """Keep generating and evaluating until requirements are met."""
    memory = []
    chain_of_thought = []
    
    thoughts, result = await generate(generator_prompt, task)
    memory.append(result)
    chain_of_thought.append({"thoughts": thoughts, "result": result})
    
    while True:
        evaluation, feedback = await evaluate(evaluator_prompt, result, task)
        if evaluation == "PASS":
            return result, chain_of_thought
            
        context = "\n".join([
            "Previous attempts:",
            *[f"- {m}" for m in memory],
            f"\nFeedback: {feedback}"
        ])
        
        thoughts, result = await generate(generator_prompt, task, context)
        memory.append(result)
        chain_of_thought.append({"thoughts": thoughts, "result": result})

In [48]:
evaluator_prompt = """
Evaluate this following code implementation for:
1. code correctness
2. time complexity
3. style and best practices

You should be evaluating only and not attemping to solve the task.
Only output "PASS" if all criteria are met and you have no further suggestions for improvements."""

generator_prompt = """
Your goal is to complete the task based on <user input>. If there are feedback 
from your previous generations, you should reflect on them to improve your solution."""

task = """
<user input>
Implement a Stack with:
1. push(x)
2. pop()
3. getMin()
All operations should be O(1).
</user input>
"""

result, chain_of_thought = await loop(task, evaluator_prompt, generator_prompt)

print("=== FINAL RESULT ===")
print(result)

print("\n=== CHAIN OF THOUGHT ===")
pprint(chain_of_thought)


=== GENERATION START ===
Thoughts:
The task is to implement a Stack data structure that supports push, pop, and getMin operations, all in constant time (O(1)).
To achieve O(1) complexity for the getMin operation, I can utilize an auxiliary stack that keeps track of the minimum values. This auxiliary stack is updated during each push and pop operation to ensure it always holds the current minimum value at the top. This way, getMin can simply return the top element of the auxiliary stack.
By revisiting past feedback, I realize the importance of ensuring clarity, efficiency, and edge case handling in my implementation. Ensuring that operations handle scenarios such as popping from an empty stack gracefully is critical.

Generated:
```python
class MinStack:
    def __init__(self):
        # Main stack to store all elements
        self.stack = []
        # Auxiliary stack to store minimum elements
        self.min_stack = []

    def push(self, x):
        # Push the element onto the main